In [ ]:
import os
from itertools import islice

import joblib

In [2]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [1]:
%load_ext autoreload
%autoreload 2
from utils.streams import document_stream, stream_cleaned_texts, reservoir_sample
from utils.topic import create_topic_model

Code running in a notebook, loading display tools


In [3]:
DATA_PATH = "/work/netarkivet-cleaned/"
texts = stream_cleaned_texts(data_path=DATA_PATH, filter_porn=False)

In [ ]:
#I don't know what the optimal number of workers is, try different things and see what works best
#2 tends to be okay.
documents = document_stream(texts, workers=2)
#Randomly sample 100_000 documents  from the first 5 million
#This way we can be sure that all topics are included,
#but we won't have to use as much memory
documents = islice(documents, 5_000_000)
documents = reservoir_sample(documents, 100_000)

In [ ]:
MODEL_TYPE = "nmf"
N_TOPICS = 100
model, matrix, vectorizer = create_topic_model(
    documents,
    model_type=MODEL_TYPE,
    n_topics=N_TOPICS,
    max_freq=0.3,
    max_vocab=15_000
)

In [ ]:
#Display topic model
pyLDAvis.sklearn.prepare(model, matrix, vectorizer)

In [ ]:
SAVE_PATH = "/work/topic_model/"
joblib.dump(model, os.path.join(SAVE_PATH, f"{MODEL_TYPE}_{N_TOPICS}.joblib"))
joblib.dump(vectorizer, os.path.join(SAVE_PATH, f"tf-idf_{MODEL_TYPE}_{N_TOPICS}.joblib"))